In [1]:
!pip install bpemb

ERROR: Could not find a version that satisfies the requirement bpemb (from versions: none)
ERROR: No matching distribution found for bpemb


In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
#Python Imports
import json
from pathlib import Path
import numpy as np
import pandas as pd

In [7]:
#Path to dataset
path_dataset = Path("D:\\Study\\Courses\\Web Science\\web_science_dataset.jsonl")
if not path_dataset.is_file():
    print("File path incorrect!")

In [35]:
#Load the dataset into a list
dataset_list = []
with open(path_dataset, 'r') as file:
    lines = file.readlines()
    for line in lines:
        data = json.loads(line)
        dataset_list.append(data)

In [36]:
#Convert data from list of dicts to dataframe
data_df = pd.DataFrame.from_records(dataset_list)

In [37]:
data_df.head()
print(data_df['question'].values.shape)

(1066,)


In [38]:
#Save the datafram to the csv
data_df.to_csv("_dataset.csv", sep='\t', encoding='utf-8')

In [39]:
#List the collumn headers
list(data_df.columns.values)

['answer',
 'answerId',
 'answerUrl',
 'category',
 'categoryId',
 'question',
 'questionId',
 'questionUrl']

In [40]:
#Compute data statistics
def get_stats(x, Type):
    sentence_length = [len(sentence) for sentence in x]
    avg_length = np.mean(sentence_length)
    std = np.std(sentence_length)
    return {Type + "_mean":avg_length, Type + "_std":std}

#Group data {Questions and Answers} by category
grouped = data_df.groupby(['categoryId'])

data_stats = []
#Iterate over the groups
for category,group in grouped:
    
    #Extract questions and answers
    question_list = group['question'].values
    answer_list = group['answer'].values
    
    #Get question and answer stats
    row_dict = {}
    row_dict.update(get_stats(question_list, "Ques"))
    row_dict.update(get_stats(answer_list, "Ans"))
    row_dict['Category'] = category
    
    #Save the stats
    data_stats.append(row_dict)

#Compute the statistics matrix
df_statistics = pd.DataFrame.from_records(data_stats)
print(df_statistics)

      Ans_mean      Ans_std  Category  Ques_mean   Ques_std
0  2105.133838  2226.671978         0  55.603535  19.918188
1  2624.635417  2039.914225         1  68.312500  22.117731
2  2711.903614  2603.750532         2  56.656627  19.756565
3  2025.311828  2244.153484         3  58.107527  19.494395
4  2634.211712  2295.137713         4  58.081081  21.972192


In [69]:
#Using the Vectorizer, convert the data into numeric vectors
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()

#Retrieve all questions
ques_list = data_df['question'].values
ques_id = data_df['categoryId'].values

# Split dataset
ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
ss.get_n_splits(ques_list, ques_id)

X_train = y_train = y_test = X_test = None
for train_index, test_index in ss.split(ques_list, ques_id):
    X_train, X_test = ques_list[train_index], ques_list[test_index]
    y_train, y_test = ques_id[train_index], ques_id[test_index]


#Apply the vectorizer on the question list
question_vectors = vectorizer.fit_transform(X_train)
_test_question_vectors = vectorizer.transform(X_test)

In [70]:
#Now, we apply the Multinomial Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# classifier
clf = MultinomialNB()
    
# Train the model
clf.fit(question_vectors, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clf_forest = RandomForestClassifier(n_estimators = 100)
clf_forest.fit(question_vectors, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [72]:
#Measure the accuracy
print('Naive Bayes',clf.score(_test_question_vectors, y_test))
print('Random Forest', clf_forest.score(_test_question_vectors, y_test))


Naive Bayes 0.6074766355140186
Random Forest 0.5841121495327103


In [73]:
#Create a Dataloader class
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import csv
import gzip

In [74]:
import torch
import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import Adam
from typing import List, Tuple

In [75]:
seed = 1000
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define some globals
batch_size = 10
lr = 2e-4
lstm_dim = 200
n_epochs = 10
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")

In [76]:
def text_to_batch_bilstm(text, tokenizer):
    """
    Creates a tokenized batch for input to a bilstm model
    :param text: A list of sentences to tokenize
    :param tokenizer: A tokenization function to use (i.e. fasttext)
    :return: Tokenized text as well as the length of the input sequence
    """
    # Some light preprocessing
    input_ids = [tokenizer.encode_ids_with_eos(t) for t in text]
    return input_ids, [len(ids) for ids in input_ids]

def collate_batch_bilstm(input_data):
    input_ids = [i[0][0] for i in input_data]
    seq_lens = [i[1][0] for i in input_data]
    labels = [i[2] for i in input_data]

    max_length = max([len(i) for i in input_ids])

    input_ids = [(i + [25000] * (max_length - len(i))) for i in input_ids]

    assert (all(len(i) == max_length for i in input_ids))
    return torch.tensor(input_ids), torch.tensor(seq_lens), torch.tensor(labels)

class SentenceDataset(Dataset):
    
    """ Load the Sentence Dataset """
    def __init__(self, input_df, tokenizer):
        
        #Retrieve all questions and labels
        self.questions = input_df['question'].values
        self.labels = input_df['categoryId'].values
        
        #Store the dataset length
        self.len = len(self.labels)
        self.tokenizer = tokenizer
    
    def __getitem__(self, index):
        
        # Select the corresponding index
        question = self.questions[index]
        label = self.labels[index]
        
        # Tokenise the question text
        input_ids, seq_lens = text_to_batch_bilstm([question], self.tokenizer)
        return input_ids, seq_lens, self.labels[index]
    
    def __len__(self):
        return self.len

In [77]:
# Load the word embeddings
from bpemb import BPEmb
bpemb_en = BPEmb(lang='en', dim=300, vs=25000)

#Add 0 embedding for padding
pretrained_embeddings = np.concatenate([bpemb_en.emb.vectors, np.zeros(shape=(1,300))], axis=0)
vocabulary = bpemb_en.emb.index2word + ['[PAD]']

In [78]:
# Split the data
train_df, test_df = train_test_split(data_df, stratify=data_df['categoryId'].values, test_size=0.2)
test_df, val_df = train_test_split(test_df, stratify=test_df['categoryId'].values, test_size=0.5)

In [79]:
# Load the train_dataset
train_dataset = SentenceDataset(train_df, bpemb_en)
train_dl = DataLoader(dataset=train_dataset, batch_size=10, shuffle=True, collate_fn=collate_batch_bilstm)

# Load the validation_dataset
val_dataset = SentenceDataset(val_df, bpemb_en)
valid_dl = DataLoader(dataset=val_dataset, batch_size=10, shuffle=True, collate_fn=collate_batch_bilstm)

In [80]:
class BiLSTMNetwork(nn.Module):

    def __init__(
            self,
            pretrained_embeddings: torch.tensor,
            lstm_dim: int,
            dropout_prob: float = 0.1,
            n_classes: int = 2
    ):
        super(BiLSTMNetwork, self).__init__()
        
        # Padding index is last in the array. Therefore we set padding_idx
        self.model = nn.ModuleDict({
            'embeddings': nn.Embedding.from_pretrained(pretrained_embeddings, padding_idx=pretrained_embeddings.shape[0] - 1),
            'bilstm': nn.LSTM(
                pretrained_embeddings.shape[1],
                lstm_dim,
                2,
                batch_first=True,
                dropout=dropout_prob,
                bidirectional=True),
            'ff': nn.Linear(2*lstm_dim, n_classes)
        })
        self.n_classes = n_classes
        self._init_weights()

    def _init_weights(self):
        all_params = list(self.model['bilstm'].named_parameters()) + \
                     list(self.model['ff'].named_parameters())
        for n,p in all_params:
            if 'weight' in n:
                nn.init.xavier_normal_(p)
            elif 'bias' in n:
                nn.init.zeros_(p)

    def forward(self, inputs, input_lens, labels = None):
        # Get embeddings (b x sl x edim)
        embeds = self.model['embeddings'](inputs)

        # Pack padded
        lstm_in = nn.utils.rnn.pack_padded_sequence(
            embeds,
            input_lens,
            batch_first=True,
            enforce_sorted=False
        )
        lstm_out, hidden = self.model['bilstm'](lstm_in)
        # Unpack (b x sl x 2*lstm_dim)
        lstm_out,_ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        # Get the last output for classification (b x 2*lstm_dim)
        ff_in = lstm_out.gather(1, input_lens.view(-1,1,1).expand(lstm_out.size(0), 1, lstm_out.size(2)) - 1).squeeze()

        # Get logits (b x 2)
        logits = self.model['ff'](ff_in).view(-1, self.n_classes)
        outputs = (logits,)
        if labels is not None:
            # Xentropy loss
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            outputs = (loss,) + outputs

        return outputs
        

In [81]:
# Create the model
model = BiLSTMNetwork(
    pretrained_embeddings=torch.FloatTensor(pretrained_embeddings), 
    lstm_dim=lstm_dim, 
    dropout_prob=0.1, 
    n_classes=len(set(data_df['categoryId'].values))
  ).to(device)

In [82]:
def accuracy(logits, labels):
  return torch.sum(torch.argmax(logits, dim=-1) == labels).type(torch.float) / float(labels.shape[0])

def train(model, train_dl, valid_dl, optimizer, n_epochs, device):
  losses = []
  best_acc = 0.0
  for ep in range(n_epochs):
    loss_epoch = []
    for batch in tqdm(train_dl):
      model.train()
      optimizer.zero_grad()
      batch = tuple(t.to(device) for t in batch)
      input_ids = batch[0]
      seq_lens = batch[1]
      labels = batch[2]

      loss, logits = model(input_ids, seq_lens, labels=labels)
      losses.append(loss.item())
      loss_epoch.append(loss.item())
      
      loss.backward()
      #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()
      #gc.collect()
    # Evaluate
    model.eval()
    with torch.no_grad():
      for batch in valid_dl:
        batch = tuple(t.to(device) for t in batch)
        input_ids = batch[0]
        seq_lens = batch[1]
        labels = batch[2]

        _, logits = model(input_ids, seq_lens, labels=labels)
        acc = accuracy(logits, labels)
        print(f'Validation accuracy: {acc}, train loss: {sum(loss_epoch) / len(loss_epoch)}')
        best_model = model.state_dict()
        if acc > best_acc:
          #best_model = model.state_dict()
          best_acc = acc
        #gc.collect()

  model.load_state_dict(best_model)
  return model, losses

In [83]:
# Create the optimizer
optimizer = Adam(model.parameters(), lr=lr)

# Train
model, losses = train(model, train_dl, valid_dl, optimizer, n_epochs, device)

D:\Anaconda\envs\dl\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



Validation accuracy: 0.6000000238418579, train loss: 1.372222630783569
Validation accuracy: 0.6000000238418579, train loss: 1.372222630783569
Validation accuracy: 0.800000011920929, train loss: 1.372222630783569
Validation accuracy: 0.6000000238418579, train loss: 1.372222630783569
Validation accuracy: 0.6000000238418579, train loss: 1.372222630783569
Validation accuracy: 0.5, train loss: 1.372222630783569
Validation accuracy: 0.699999988079071, train loss: 1.372222630783569
Validation accuracy: 0.800000011920929, train loss: 1.372222630783569
Validation accuracy: 0.699999988079071, train loss: 1.372222630783569
Validation accuracy: 1.0, train loss: 1.372222630783569
Validation accuracy: 1.0, train loss: 1.372222630783569



Validation accuracy: 0.699999988079071, train loss: 0.8419690236102703
Validation accuracy: 0.6000000238418579, train loss: 0.8419690236102703
Validation accuracy: 1.0, train loss: 0.8419690236102703
Validation accuracy: 0.6000000238418579, train loss: 0.8419690236102703
Validation accuracy: 0.800000011920929, train loss: 0.8419690236102703
Validation accuracy: 0.800000011920929, train loss: 0.8419690236102703
Validation accuracy: 0.9000000357627869, train loss: 0.8419690236102703
Validation accuracy: 0.800000011920929, train loss: 0.8419690236102703
Validation accuracy: 0.9000000357627869, train loss: 0.8419690236102703
Validation accuracy: 0.9000000357627869, train loss: 0.8419690236102703
Validation accuracy: 0.5714285969734192, train loss: 0.8419690236102703



Validation accuracy: 0.699999988079071, train loss: 0.6412608724000842
Validation accuracy: 0.4000000059604645, train loss: 0.6412608724000842
Validation accuracy: 0.9000000357627869, train loss: 0.6412608724000842
Validation accuracy: 0.699999988079071, train loss: 0.6412608724000842
Validation accuracy: 0.4000000059604645, train loss: 0.6412608724000842
Validation accuracy: 0.6000000238418579, train loss: 0.6412608724000842
Validation accuracy: 0.699999988079071, train loss: 0.6412608724000842
Validation accuracy: 0.800000011920929, train loss: 0.6412608724000842
Validation accuracy: 0.800000011920929, train loss: 0.6412608724000842
Validation accuracy: 0.699999988079071, train loss: 0.6412608724000842
Validation accuracy: 0.8571429252624512, train loss: 0.6412608724000842



Validation accuracy: 0.800000011920929, train loss: 0.5042707013875939
Validation accuracy: 0.800000011920929, train loss: 0.5042707013875939
Validation accuracy: 0.699999988079071, train loss: 0.5042707013875939
Validation accuracy: 0.9000000357627869, train loss: 0.5042707013875939
Validation accuracy: 1.0, train loss: 0.5042707013875939
Validation accuracy: 0.800000011920929, train loss: 0.5042707013875939
Validation accuracy: 0.800000011920929, train loss: 0.5042707013875939
Validation accuracy: 0.800000011920929, train loss: 0.5042707013875939
Validation accuracy: 0.6000000238418579, train loss: 0.5042707013875939
Validation accuracy: 0.6000000238418579, train loss: 0.5042707013875939
Validation accuracy: 0.8571429252624512, train loss: 0.5042707013875939


...


Validation accuracy: 0.800000011920929, train loss: 0.287609976044921
Validation accuracy: 0.9000000357627869, train loss: 0.287609976044921
Validation accuracy: 0.9000000357627869, train loss: 0.287609976044921
Validation accuracy: 0.6000000238418579, train loss: 0.287609976044921
Validation accuracy: 0.699999988079071, train loss: 0.287609976044921
Validation accuracy: 0.9000000357627869, train loss: 0.287609976044921
Validation accuracy: 0.800000011920929, train loss: 0.287609976044921
Validation accuracy: 0.699999988079071, train loss: 0.287609976044921
Validation accuracy: 0.699999988079071, train loss: 0.287609976044921
Validation accuracy: 0.800000011920929, train loss: 0.287609976044921
Validation accuracy: 0.7142857313156128, train loss: 0.287609976044921



Validation accuracy: 0.699999988079071, train loss: 0.26584671883908817
Validation accuracy: 0.699999988079071, train loss: 0.26584671883908817
Validation accuracy: 0.9000000357627869, train loss: 0.26584671883908817
Validation accuracy: 0.800000011920929, train loss: 0.26584671883908817
Validation accuracy: 0.9000000357627869, train loss: 0.26584671883908817
Validation accuracy: 0.699999988079071, train loss: 0.26584671883908817
Validation accuracy: 0.9000000357627869, train loss: 0.26584671883908817
Validation accuracy: 0.6000000238418579, train loss: 0.26584671883908817
Validation accuracy: 0.800000011920929, train loss: 0.26584671883908817
Validation accuracy: 0.9000000357627869, train loss: 0.26584671883908817
Validation accuracy: 1.0, train loss: 0.26584671883908817



Validation accuracy: 0.9000000357627869, train loss: 0.2128317209697047
Validation accuracy: 0.6000000238418579, train loss: 0.2128317209697047
Validation accuracy: 0.9000000357627869, train loss: 0.2128317209697047
Validation accuracy: 0.5, train loss: 0.2128317209697047
Validation accuracy: 0.9000000357627869, train loss: 0.2128317209697047
Validation accuracy: 0.6000000238418579, train loss: 0.2128317209697047
Validation accuracy: 0.9000000357627869, train loss: 0.2128317209697047
Validation accuracy: 0.800000011920929, train loss: 0.2128317209697047
Validation accuracy: 0.9000000357627869, train loss: 0.2128317209697047
Validation accuracy: 0.800000011920929, train loss: 0.2128317209697047
Validation accuracy: 0.7142857313156128, train loss: 0.2128317209697047



Validation accuracy: 0.6000000238418579, train loss: 0.15766642623862556
Validation accuracy: 0.800000011920929, train loss: 0.15766642623862556
Validation accuracy: 0.800000011920929, train loss: 0.15766642623862556
Validation accuracy: 1.0, train loss: 0.15766642623862556
Validation accuracy: 0.699999988079071, train loss: 0.15766642623862556
Validation accuracy: 0.9000000357627869, train loss: 0.15766642623862556
Validation accuracy: 0.6000000238418579, train loss: 0.15766642623862556
Validation accuracy: 0.800000011920929, train loss: 0.15766642623862556
Validation accuracy: 0.6000000238418579, train loss: 0.15766642623862556
Validation accuracy: 0.9000000357627869, train loss: 0.15766642623862556
Validation accuracy: 0.8571429252624512, train loss: 0.15766642623862556



Validation accuracy: 0.9000000357627869, train loss: 0.12897106530253105
Validation accuracy: 0.699999988079071, train loss: 0.12897106530253105
Validation accuracy: 0.9000000357627869, train loss: 0.12897106530253105
Validation accuracy: 0.800000011920929, train loss: 0.12897106530253105
Validation accuracy: 0.9000000357627869, train loss: 0.12897106530253105
Validation accuracy: 0.800000011920929, train loss: 0.12897106530253105
Validation accuracy: 0.699999988079071, train loss: 0.12897106530253105
Validation accuracy: 0.800000011920929, train loss: 0.12897106530253105
Validation accuracy: 0.800000011920929, train loss: 0.12897106530253105
Validation accuracy: 0.800000011920929, train loss: 0.12897106530253105
Validation accuracy: 0.4285714626312256, train loss: 0.12897106530253105
